In [1]:
# default_exp util

In [2]:
%pip install nbdev==1.2.11 | grep -v 'already satisfied'

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
sys.path.append('..')
import lib_py.ipython_magic as ipython_magic

# spiral

In [4]:
%%spi

nominal _obj = $"obj"

inl test b = assert b "test"


<magic:aa36b2e1d1cc4ec6b7e715e8b1eefae9>


<IPython.core.display.Javascript object>

'/workspaces/spiral-notebook/lib_spi/util.spi'

# node

In [5]:
%%node export

export type PickByType<T, Value> = {
    [P in keyof T as T[P] extends Value | undefined ? P : never]: T[P]
}

type NoneEmptyArray = readonly any[] & { 0: any }
type CompareUnionWithArray<P, Q extends NoneEmptyArray> = Exclude<P, Q[number]> extends never
    ? (Exclude<Q[number], P> extends never ? Q : ReadonlyArray<P>)
    : readonly [...Q, Exclude<P, Q[number]>]
export function assertTypeEquals<P, Q extends NoneEmptyArray>(_test: CompareUnionWithArray<P, Q>): void { }

'/workspaces/spiral-notebook/lib_ts/util.ts'

In [6]:
%%node export

import * as fs from "fs"

var start = process.hrtime()

export var elapsed = (start: [number, number]) => process.hrtime(start)[1] / 1000000
export var logStep = (note: string) => {
    console.log(`${process.hrtime(start)[0]} s, ${elapsed(start).toFixed(3)} ms - ${note}`)
    start = process.hrtime()
}
export var sleep = (ms: number) => new Promise(resolve => setTimeout(resolve, ms))
export var waitFileChange = async (path: string) => {
    var start = process.hrtime()
    const stat_mtime = fs.statSync(path).mtimeMs
    const oldLines = fs.readFileSync(path).toString()
    let newLines = oldLines
    while (elapsed(start) < 30000
        && (fs.statSync(path).mtimeMs === stat_mtime
            || newLines === oldLines
            || newLines.length === 0)) {
        await sleep(50)
        newLines = fs.readFileSync(path).toString()
    }
    return newLines
}


'/workspaces/spiral-notebook/lib_ts/util.ts'

In [7]:
%%node export

export var timeout = async <T>(prom: Promise<T>, ms: number) => {
	let timer
	try {
        return await Promise.race([
            prom,
            new Promise((_r, rej) => timer = setTimeout(rej, ms, new Error('timeout')))
        ])
    } finally {
        return clearTimeout(timer)
    }
}


'/workspaces/spiral-notebook/lib_ts/util.ts'

# python

In [8]:
#export

import uuid

def new_id():
    return uuid.uuid4().hex

def list_partition(lines, predicate):
    l1, l2 = [], []
    for line in lines:
        (l1, l2)[not predicate(line)].append(line)
    return l1, l2

def write_file(filename, text):
    with open(filename, 'w') as f:
        return f.write(text)

def read_file(filename):
    with open(filename, 'r') as f:
        return f.read()


In [9]:
#export

from time import sleep
from IPython.display import display, Javascript
import subprocess
import os
import uuid
import re

def get_notebook_name():
    if '__vsc_ipynb_file__' in globals():
        ipynb_path = __vsc_ipynb_file__
    else:
        magic = new_id()
        print(f'<magic:{magic}>')

        display(Javascript('IPython.notebook.save_checkpoint();'))

        nb_name = None
        while nb_name is None:
            try:
                sleep(0.1)
                nb_name = subprocess.check_output(f'grep -l {magic} *.ipynb', shell=True).decode().strip()
            except:
                pass

        ipynb_path = os.path.join(os.getcwd(), nb_name)

    ipynb_body = read_file(ipynb_path)
    return re.search(r'\# default_exp (\w+) *', ipynb_body).group(1)

_notebook_name = None


In [10]:
_notebook_name = get_notebook_name()
_notebook_name

'util'

In [11]:
#export

import subprocess

def run_shell(*args, **kwargs):
    try:
        return subprocess.run(*args, shell=True, capture_output=True, **kwargs)
    except Exception as e:
        print(f'util.run_shell() error. args={args} kwargs={kwargs} e={e}')
        raise e
    
def run(*args, **kwargs):
    shell = run_shell(*args, **kwargs)
    err = shell.stderr.decode()
    if err != '':
        print(f'util.run() error. args={args} kwargs={kwargs} err={err}')
        raise Exception(err)
    return shell.stdout.decode().strip('\n')

def run_node(code, timeout=60, *args, **kwargs):
    e = ' --input-type=module -e ' if '\n' in code or '.' not in code else ''
    return run(
        f'''NODE_NO_WARNINGS=1 node --loader ts-node/esm --es-module-specifier-resolution=node {e} \'{code}\'''', 
        *args, 
        timeout=timeout, 
        **kwargs
    )


# py compile

In [12]:
!nbdev_build_lib

Converted 00_util.ipynb.
Converted 01_spiral_api.ipynb.
Converted 02_ipython_magic.ipynb.
Converted 03_tictactoe_pyteal.ipynb.
Converted 04_algo_network.ipynb.
Converted 05_tictactoe_testnet.ipynb.
Converted 06_index.ipynb.


Bad pipe message: %s [b'\xaaw\xef\xbd~\xe7=\xe8u\xb7>\x12\xeb\xb5\xcc\xc2\x8c\xbe \x12Aj\xfc\x05\xc0;\x13\xc8\xd3?c\x14y0T8\xfe\xf4\x83nD\xf3hX)\xf1\xaf\x1c\xd7X*\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06', b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b')\x1a\x9c\xbdE\xb3\x04\x1cM\x9d\xd0P\x1d\xbciE\xd8t \xfd\x03^\xbe\xc9D?`y1\x1c\x1b\xcd\x8eQ\x05\xa4\xa2lj\x00\xceE]\xcc\x8ey\xff\x07\xf3\xe8\xc5\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00']
Bad pipe message: %s [b"\x8eY\x90\x8fPV*c\xee\x08\xf3p\xc5?\x0e3P2\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae